In [1]:
import requests
from bs4 import BeautifulSoup
from sys import stdout
import numpy as np
import re
import pandas as pd
import time

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [3]:
page_list = []
rows = 0
while rows < 90:
    r = requests.get('https://auto.8891.com.tw/usedauto-moto.html?firstRow=' + str(rows), headers=headers)
    
    if r.status_code != 200:
        print("\nERROR on row", rows)
        continue
    
    soup = BeautifulSoup(r.content, 'lxml')
    divs = soup.find_all("div", {'class':'text-box'})
    for a in divs:
        page_list.append(a.a.get('href'))

    lis = soup.find_all("li", {'class': 'ItemTitle'})
    for a in lis:
        page_list.append(a.a.get('href'))
        
    rows += 30
    stdout.write('\rprocess rows: %d' % (rows))
    stdout.flush()
        

process rows: 90

In [4]:
# 爬圖片
def save_img(img_name, url):
    if url == 'https://img.8891.com.tw/v3/infos/no-picture-big.png':
        return -1
    for i in range(5):
        try:
            img_data = requests.get(url).content
            with open('./DEMO/' + img_name + '.jpg', 'wb') as handler:
                handler.write(img_data)
            break
        except:
            print("Connection Error", url)
            time.sleep(10)
    return 0

In [5]:
df = pd.DataFrame(columns=['pic_name', 'price'])
for page in page_list:
    page_id = re.findall(r"[\w']+", page)[2]
    r = requests.get('https://auto.8891.com.tw' + page, headers=headers)
    if r.status_code != 200:
        print("\ncan't connect")
        continue
    soup = BeautifulSoup(r.content, 'lxml')
    ul = soup.find("ul", {"id":'thumb-list'})
    if not ul:
        print("\nno thumb-list")
        continue
    imgs = ul.find_all('img')
    
    price = -1
    s = soup.find("dd", {"class":"value Blod F15px Red"})
    try:
        price = float(s.getText().strip()[:-1])
    except:
        print("\nno price")
        continue
    
    pic_count = 0
    for a in imgs:
        pic_name = page_id + "_" + str(pic_count)
        if save_img(pic_name, a.get("_src")) == -1:
            print("\nno picture")
            break
        stdout.write('\rprocess rows: %s       ' % (pic_name))
        stdout.flush()
        df = df.append({'pic_name':pic_name, 'price': price}, ignore_index=True)
        pic_count += 1

process rows: 1649607_5        
no picture
process rows: 1644713_0        

In [6]:
df.to_csv('price_test.csv', index=False)

In [7]:
df

,pic_name,price
0,1624381_0,3.0
1,1624381_1,3.0
2,1624381_2,3.0
3,1624381_3,3.0
4,1624381_4,3.0
5,1624381_5,3.0
6,1624381_6,3.0
7,1624381_7,3.0
8,1624381_8,3.0
9,1624381_9,3.0
